# Chemical Reactions: The Reacter Module

Imagine if you could **program** chemical reactions like building with LEGO blocks 🧱

That's exactly what `molpy.reacter` lets you do! ✨

## Why Reacter?

In molecular simulation, we frequently need to:

- 🧪 Build polymers (linking monomers)
- 🔗 Simulate chemical reactions (breaking/forming bonds)
- 🧬 Generate complex molecular structures

Traditional approaches either rely on external tools (like RDKit's SMIRKS) or require writing complex atom manipulation code from scratch.

**Reacter's Design Philosophy:**

- **Pure Python** - No RDKit dependency, fully native molpy data structures
- **Composable** - Reaction logic = modular function composition
- **Auditable** - All changes recorded in `ProductSet.notes`
- **Stable Indexing** - Atom deletion doesn't scramble IDs

---

## Core Concept: Reactions as Recipes

Think of a `Reacter` as a **chemical recipe** composed of three key ingredients:

```
┌─────────────────────────────────────────────┐
│           Reacter (Reaction)                │
│                                             │
│  1️⃣  Selectors                              │
│      ├─ Find reaction sites (anchors)       │
│      └─ Find atoms to remove (leaving grps) │
│                                             │
│  2️⃣  Transformers                           │
│      └─ Create new chemical bonds           │
│                                             │
│  3️⃣  ProductSet                             │
│      ├─ Reaction product                    │
│      └─ Metadata (removed atoms, new bonds) │
└─────────────────────────────────────────────┘
```

Let's dive into a real chemical reaction to understand these concepts!

---

## Your First Reaction: Esterification ⭐

We'll simulate a classic organic chemistry reaction: **acetic acid + ethanol → ethyl acetate**

```
CH₃COOH  +  CH₃CH₂OH  →  CH₃COOCH₂CH₃  +  H₂O
(acetic acid) (ethanol)   (ethyl acetate)  (water)
```

The reaction mechanism:
- Acetic acid loses an **-OH** group
- Ethanol loses an **-H**
- They connect via a **C-O** ester bond
- The leaving OH and H combine to form water

### Step 1: Build the Reactants

First, let's construct the molecular structures of acetic acid and ethanol:

In [ ]:
from molpy import Atom, Bond
from molpy.core.wrappers.monomer import Monomer


def create_acetic_acid():
    """Create acetic acid: CH3-COOH"""
    mono = Monomer(name="acetic_acid")

    # CH3 methyl group
    c1 = Atom(symbol="C", name="C1")
    h1 = Atom(symbol="H", name="H1")
    h2 = Atom(symbol="H", name="H2")
    h3 = Atom(symbol="H", name="H3")

    # COOH carboxyl group
    c2 = Atom(symbol="C", name="C2")  # Carbonyl carbon
    o1 = Atom(symbol="O", name="O1")  # C=O double-bonded oxygen
    o2 = Atom(symbol="O", name="O2")  # C-OH hydroxyl oxygen
    h4 = Atom(symbol="H", name="H4")  # Hydroxyl hydrogen

    mono.add_entity(c1, h1, h2, h3, c2, o1, o2, h4)

    # Bonds
    mono.add_link(
        Bond(c1, h1, order=1),
        Bond(c1, h2, order=1),
        Bond(c1, h3, order=1),
        Bond(c1, c2, order=1),  # CH3-C
        Bond(c2, o1, order=2),  # C=O (carbonyl)
        Bond(c2, o2, order=1),  # C-OH (hydroxyl)
        Bond(o2, h4, order=1),  # O-H
    )

    # Set port: carbonyl carbon is the reaction site
    mono.set_port("1", c2)

    return mono


def create_ethanol():
    """Create ethanol: CH3-CH2-OH"""
    mono = Monomer(name="ethanol")

    # CH3 methyl group
    c1 = Atom(symbol="C", name="C1")
    h1 = Atom(symbol="H", name="H1")
    h2 = Atom(symbol="H", name="H2")
    h3 = Atom(symbol="H", name="H3")

    # CH2OH hydroxymethyl group
    c2 = Atom(symbol="C", name="C2")
    h4 = Atom(symbol="H", name="H4")
    h5 = Atom(symbol="H", name="H5")
    o = Atom(symbol="O", name="O")
    h6 = Atom(symbol="H", name="H6")  # This H will be removed

    mono.add_entity(c1, h1, h2, h3, c2, h4, h5, o, h6)

    # Bonds
    mono.add_link(
        Bond(c1, h1, order=1),
        Bond(c1, h2, order=1),
        Bond(c1, h3, order=1),
        Bond(c1, c2, order=1),  # CH3-CH2
        Bond(c2, h4, order=1),
        Bond(c2, h5, order=1),
        Bond(c2, o, order=1),  # CH2-O
        Bond(o, h6, order=1),  # O-H
    )

    # Set port: oxygen is the reaction site
    mono.set_port("1", o)

    return mono


# Create the reactants
acetic_acid = create_acetic_acid()
ethanol = create_ethanol()

print(f"Acetic acid: {len(list(acetic_acid.atoms))} atoms")
print(f"Ethanol: {len(list(ethanol.atoms))} atoms")

### Step 2: Define the Esterification Reaction

Now we assemble the "reaction recipe":

In [ ]:
from molpy.reacter import (
    Reacter,
    port_anchor_selector,  # Find reaction sites via ports
    remove_OH,  # Remove -OH group
    remove_one_H,  # Remove one hydrogen
    make_single_bond,  # Create single bond
)

# Create esterification reacter
esterification = Reacter(
    name="esterification",
    # Left reactant: acetic acid
    anchor_left=port_anchor_selector,  # Find carbonyl carbon via port
    leaving_left=remove_OH,  # Remove -OH group
    # Right reactant: ethanol
    anchor_right=port_anchor_selector,  # Find oxygen via port
    leaving_right=remove_one_H,  # Remove -H
    # Bond formation
    bond_maker=make_single_bond,  # Form C-O ester bond
)

print(f"Reacter: {esterification}")
print("Reaction mechanism:")
print("  Acetic acid -OH  →  (removed)")
print("  Ethanol -H       →  (removed)")
print("  C + O            →  C-O ester bond")

### Step 3: Run the Reaction!

Executing the reaction is as simple as pressing "start":

In [ ]:
# Execute the reaction
product = esterification.run(
    acetic_acid,
    ethanol,
    port_L="1",  # Acetic acid port
    port_R="1",  # Ethanol port
)

print("=" * 50)
print("Reaction Complete!")
print("=" * 50)

# Inspect the product
print(f"\nProduct atoms: {len(list(product.product.atoms))}")
print(f"Removed atoms: {product.notes['removed_count']}")
print(f"New bonds: {len(product.notes['new_bonds'])}")

# Calculate molecular formula
atom_counts = {}
for atom in product.product.atoms:
    symbol = atom.get("symbol")
    atom_counts[symbol] = atom_counts.get(symbol, 0) + 1

formula = "".join(
    f"{symbol}{atom_counts[symbol] if atom_counts[symbol] > 1 else ''}"
    for symbol in sorted(atom_counts.keys())
    if symbol
)

print(f"\nProduct formula: {formula}")
print("Expected formula: C4H8O2 (ethyl acetate)")

if formula == "C4H8O2":
    print("\n🎉 Success! Ethyl acetate formed!")
else:
    print(f"\n⚠️  Formula mismatch, check reaction")

### Understanding the ProductSet

`ProductSet` contains not just the product, but a complete "reaction log":

In [ ]:
print("Reaction Details:")
print(f"  Reaction name: {product.notes['reaction_name']}")
print(f"  Left port: {product.notes['port_L']}")
print(f"  Right port: {product.notes['port_R']}")
print(f"  Removed atoms: {product.notes['removed_count']}")
print(f"  New bonds: {len(product.notes['new_bonds'])}")
print(f"  Needs retypification: {product.notes['needs_retypification']}")

print("\n💡 Tip:")
print("  - removed_atoms contains the removed atom entities")
print("  - anchor_left/right records the reaction sites")
print("  - This metadata is invaluable for debugging!")

---

## Selector Catalog: Finding the Right Atoms

Selectors handle two jobs:
1. **Find anchors** - where the reaction occurs
2. **Find leaving groups** - which atoms to remove

### Anchor Selectors

#### `port_anchor_selector` - The Standard

The most common selector, finds reaction sites via ports:

In [ ]:
from molpy.reacter import port_anchor_selector

# We already set ports in our example:
# acetic_acid.set_port("1", c2)  # Carbonyl carbon
# ethanol.set_port("1", o)        # Oxygen

# port_anchor_selector automatically finds these atoms
anchor = port_anchor_selector(acetic_acid, "1")
print(f"Acetic acid reaction site: {anchor.get('name')} ({anchor.get('symbol')})")

anchor = port_anchor_selector(ethanol, "1")
print(f"Ethanol reaction site: {anchor.get('name')} ({anchor.get('symbol')})")

### Leaving Group Selectors

#### `remove_one_H` - Remove One Hydrogen

In [ ]:
from molpy.reacter import remove_one_H

# Create a simple methane molecule CH4
test_mono = Monomer()
c = Atom(symbol="C")
h1 = Atom(symbol="H")
h2 = Atom(symbol="H")
h3 = Atom(symbol="H")
h4 = Atom(symbol="H")

test_mono.add_entity(c, h1, h2, h3, h4)
test_mono.add_link(Bond(c, h1), Bond(c, h2), Bond(c, h3), Bond(c, h4))

leaving = remove_one_H(test_mono, c)
print(f"remove_one_H: removes {len(leaving)} H atom(s)")
print("Use case: condensation reactions needing single H removal")

#### `remove_all_H` - Remove All Hydrogens

In [ ]:
from molpy.reacter import remove_all_H

leaving = remove_all_H(test_mono, c)
print(f"remove_all_H: removes {len(leaving)} H atoms")
print("Use case: forming unsaturated bonds (e.g., C=C double bonds)")

#### `remove_OH` - Remove Hydroxyl Group

✨ **Now works correctly for carboxylic acids!** It distinguishes between C=O (double bond) and C-OH (single bond).

In [ ]:
from molpy.reacter import remove_OH

print("remove_OH handles:")
print("  ✓ Alcohols (R-OH)")
print("  ✓ Phenols (Ar-OH)")
print("  ✓ Carboxylic acids (R-COOH) - distinguishes C=O from C-OH!")
print("\nHow it works:")
print("  - Checks bond order to oxygen neighbors")
print("  - Selects only single-bonded O (hydroxyl)")
print("  - Ignores double-bonded O (carbonyl)")

#### `no_leaving_group` - Addition Reactions

In [ ]:
from molpy.reacter import no_leaving_group

leaving = no_leaving_group(test_mono, c)
print(f"no_leaving_group: removes {len(leaving)} atoms")
print("Use case: addition reactions - no atoms removed, only bond formation")

---

## Transformer Catalog: Creating Bonds

Transformers create chemical bonds between anchor atoms.

### Basic Bond Types

In [ ]:
from molpy.reacter import (
    make_single_bond,  # C-C  (order=1)
    make_double_bond,  # C=C  (order=2)
    make_triple_bond,  # C≡C  (order=3)
    make_aromatic_bond,  # C:C  (order=1.5)
)

print("Bond transformers:")
print("  make_single_bond   → C-C  (single, order=1)")
print("  make_double_bond   → C=C  (double, order=2)")
print("  make_triple_bond   → C≡C  (triple, order=3)")
print("  make_aromatic_bond → C:C  (aromatic, order=1.5)")

### Advanced: Factory Pattern

In [ ]:
from molpy.reacter import make_bond_by_order

# Dynamically create bond makers with specific order
double_bond_maker = make_bond_by_order(2)
triple_bond_maker = make_bond_by_order(3)

print("Factory pattern: create transformers dynamically")
print("Use case: when bond order varies based on conditions")

### Special Transformers

In [ ]:
from molpy.reacter import no_new_bond, break_bond

print("Special transformers:")
print("  no_new_bond → Don't create bonds (only remove leaving groups)")
print("  break_bond  → Break existing bonds")

---

## More Reaction Examples

### Example 1: C-C Coupling (Polymer Linkage)

In [ ]:
# Create two simple C-H units
def create_ch_unit(name):
    mono = Monomer(name=name)
    c = Atom(symbol="C")
    h = Atom(symbol="H")
    mono.add_entity(c, h)
    mono.add_link(Bond(c, h))
    mono.set_port("1", c)
    return mono


mono_A = create_ch_unit("A")
mono_B = create_ch_unit("B")

# Create C-C coupling reaction
cc_coupling = Reacter(
    name="C-C_coupling",
    anchor_left=port_anchor_selector,
    anchor_right=port_anchor_selector,
    leaving_left=remove_one_H,
    leaving_right=remove_one_H,
    bond_maker=make_single_bond,
)

product = cc_coupling.run(mono_A, mono_B, port_L="1", port_R="1")
print(f"C-C coupling: {product.notes['removed_count']} H atoms removed")
print(f"Product atoms: {len(list(product.product.atoms))} (2 carbons)")

### Example 2: Asymmetric Reaction (Forming Double Bonds)

In [ ]:
# Create a carbon with multiple hydrogens
def create_ch2_unit():
    mono = Monomer()
    c = Atom(symbol="C")
    h1 = Atom(symbol="H")
    h2 = Atom(symbol="H")
    mono.add_entity(c, h1, h2)
    mono.add_link(Bond(c, h1), Bond(c, h2))
    mono.set_port("1", c)
    return mono


left = create_ch2_unit()
right = create_ch_unit("R")

# Asymmetric reaction: remove all H from left, one H from right, form double bond
double_bond_reaction = Reacter(
    name="form_double_bond",
    anchor_left=port_anchor_selector,
    anchor_right=port_anchor_selector,
    leaving_left=remove_all_H,  # Remove all H (2 atoms)
    leaving_right=remove_one_H,  # Remove one H
    bond_maker=make_double_bond,  # Form C=C
)

product = double_bond_reaction.run(left, right, port_L="1", port_R="1")
print(f"Double bond formed: {product.notes['removed_count']} H atoms removed")
print(f"Bond order: {product.notes['new_bonds'][0].get('order')} (double)")

### Example 3: Addition Reaction (No Atom Removal)

In [ ]:
def create_carbon():
    mono = Monomer()
    c = Atom(symbol="C")
    mono.add_entity(c)
    mono.set_port("1", c)
    return mono


c1 = create_carbon()
c2 = create_carbon()

# Pure addition: no atoms removed, only bond formation
addition = Reacter(
    name="addition",
    anchor_left=port_anchor_selector,
    anchor_right=port_anchor_selector,
    leaving_left=no_leaving_group,
    leaving_right=no_leaving_group,
    bond_maker=make_single_bond,
)

product = addition.run(c1, c2, port_L="1", port_R="1")
print(f"Addition: {product.notes['removed_count']} atoms removed (should be 0)")
print(f"Product atoms: {len(list(product.product.atoms))} (2 carbons)")

---

## Advanced: Tracking Reaction Intermediates

Want to see each step of the reaction? Use `record_intermediates=True`:

In [ ]:
# Re-run esterification with intermediate tracking
acetic_acid = create_acetic_acid()
ethanol = create_ethanol()

product = esterification.run(
    acetic_acid,
    ethanol,
    port_L="1",
    port_R="1",
    record_intermediates=True,  # Enable tracking!
)

# View reaction steps
if "intermediates" in product.notes:
    print("Reaction Steps:")
    for i, step in enumerate(product.notes["intermediates"], 1):
        print(f"\nStep {i}: {step['step']}")
        print(f"  {step['description']}")
        if "product" in step:
            atoms = len(list(step["product"].atoms))
            print(f"  Current atoms: {atoms}")

---

## Integration: Polymer Building

Reacter's most powerful use case is polymer construction. Via `ReacterConnector`, you can automate monomer linkage:

In [ ]:
from molpy.builder.polymer.linear import linear
from molpy.builder.polymer.connectors import ReacterConnector

# Create monomer library
library = {
    "A": create_ch_unit("A"),
    "B": create_ch_unit("B"),
}

# Create connector using C-C coupling reaction
connector = ReacterConnector(
    default_reacter=cc_coupling,  # Default to C-C coupling
)

# Build linear polymer
polymer = linear(
    sequence="ABABAB",  # Monomer sequence
    library=library,
    connector=connector,
)

print(f"Polymer built!")
print(f"Total atoms: {len(list(polymer.inner.atoms))}")
print(f"Total bonds: {len(list(polymer.inner.bonds))}")

# Tip: Use typifier for force field assignment
print("\n💡 Next step: Use OplsAtomisticTypifier for force field parameters")

---

## Best Practices & Debugging

### 1. Port Naming Conventions

```python
# ✓ Good naming
mono.set_port("head", carbon_1)  # Descriptive names
mono.set_port("tail", carbon_n)
mono.set_port("1", reactive_site)  # Numeric IDs

# ✗ Avoid
mono.set_port("x", atom)  # Unclear meaning
```

### 2. When to Retypify

After reactions, atom chemical environments change, requiring force field retypification:

In [ ]:
# Check if retypification needed
if product.notes["needs_retypification"]:
    print("⚠️  Recommend force field retypification")
    print("   from molpy.typifier.atomistic import OplsAtomisticTypifier")
    print("   typifier = OplsAtomisticTypifier()")
    print("   typifier.typify(product.product)")

### 3. Debugging Reactions

If results don't match expectations:

In [ ]:
print("Debugging checklist:")
print("\n1. Verify ports:")
print("   port_def = mono.get_port_def('1')")
print("   print(port_def.target)  # Confirm targets correct atom")

print("\n2. Check leaving groups:")
print("   leaving = remove_OH(mono, anchor)")
print("   print(f'Will remove {len(leaving)} atoms')")

print("\n3. Use record_intermediates=True to see each step")

print("\n4. Verify molecular formula:")
print("   Count atoms to confirm stoichiometry")

### 4. Performance Tips

For large-scale polymer building:

In [ ]:
print("Performance tips:")
print("  1. Avoid unnecessary record_intermediates (memory intensive)")
print("  2. Set compute_topology=False if angles/dihedrals not needed")
print("  3. For batch building, typify once at the end")
print("  4. For very large polymers (>10k atoms), consider chunked building")

---

## Summary

Congratulations! You've mastered Reacter's core concepts:

✅ **Reacter** = Selectors + Transformers  
✅ **Selectors** find reaction sites and leaving groups  
✅ **Transformers** create or break chemical bonds  
✅ **ProductSet** records the complete reaction history  
✅ **remove_OH** now correctly handles carboxylic acids  
✅ **Integration** with builder enables automated polymer synthesis  

### Next Steps

- 📖 See `user-guide/builder.ipynb` for polymer building
- 📖 See `user-guide/typifier.ipynb` for force field assignment
- 🧪 Try implementing your own chemical reactions!

### References

- API docs: `docs/api/reacter.md`
- Test cases: `tests/test_reacter/`
- Source code: `src/molpy/reacter/`

Happy coding! 🎉